In [1]:
# Install and use AMPL with any solver on Colab
!pip install -q amplpy
from amplpy import tools
ampl = tools.ampl_notebook(
    modules=['highs', 'gurobi','cplex'], 
    license_uuid="default") 

Please provide a valid license UUID. You can use a free https://ampl.com/ce license.
AMPL Version 20230124 (MSVC 19.29.30147.0, 64-bit)
Demo license with maintenance expiring 20240131.
Using license file "c:\Users\ManueleNolli\miniconda3\envs\machine-learning\lib\site-packages\ampl_module_base\bin\ampl.lic".



In [2]:
%%writefile model.mod

# Sets
set C;      # Classes
set N;      # Items
set R;      # Resources

# Parameters
param q{R};     # Resource capacity
param v{C,N};  # Value of each item in each class
param w{C,N,R};  # Weight of each item in each class and resource

# Variables
var x{C,N} binary;   # Whether each item in each class is selected

# Objective
maximize z: sum{c in C, n in N} v[c,n]*x[c,n];


# Constraints
s.t. capacity_constraint{r in R}:
  sum{c in C, n in N} w[c,n,r]*x[c,n] <= q[r];

s.t. class_constraint{c in C}:
  sum{n in N} x[c,n] = 1;

Writing model.mod


In [3]:
%%writefile data.dat

set C := 1 2 3; # Classes
set N := 1 2 3; # Items
set R := 1 2;   # Resources

# Parameters
param q := # Max capacity
  1 20
  2 26;

param v : 1 2 3 := #Volume
       1 23 13 20
       2 1 8 11
       3 3 21 14
;

param w := #Weight
# Resource 1 (columns = N = items,  rows = C = classes)
[*,*,1]: 1 2 3 :=
      1  5 2 4
      2  50 6 8
      3  1 7 8

# Resource 2
[*,*,2]: 1 2 3  :=
      1  7 9 6
      2  50 12 7
      3  1 11 12;


Writing data.dat


In [4]:
%%writefile onlineExample.dat
#https://www.researchgate.net/figure/An-instance-of-the-MMKP_fig4_220358247
set C := 1 2 3 4; # Classes
set N := 1 2 3; # Items
set R := 1 2;   # Resources

# Parameters
param q := # Max capacity
  1 26
  2 13;

param v : 1 2 3 := #Volume (columns = N = items,  rows = C = classes)
       1 60 50 19
       2 75 68 30
       3 69 57 41
       4 79 65 33
;

param w := #Weight
# Resource 1 (columns = N = items,  rows = C = classes)
[*,*,1]: 1 2 3 :=
      1  8 6 1
      2  8 9 3
      3  9 7 2
      4 10 8 4

# Resource 2
[*,*,2]: 1 2 3 :=
      1  4 3 2      
      2  5 4 2
      3  3 3 4
      4  3 4 1; 

Writing onlineExample.dat


In [5]:
%%ampl_eval

reset;

/* Tells AMPL to use CPLEX*/
option solver cplex;

option presolve 0;
option cplex_options 'presolve 0 sensitivity';

/* Display 2 digits */
option display_round 2;

model model.mod;  
data onlineExample.dat;

solve;
display z;
display x;

display "DATA";
display v;
display q;
display w;

CPLEX 22.1.1.0: presolve 0
sensitivity
CPLEX 22.1.1.0: optimal integer solution; objective 230
4 MIP simplex iterations
0 branch-and-bound nodes

suffix up OUT;
suffix down OUT;
suffix current OUT;
z = 230.00

x :=
1 1   0.00
1 2   0.00
1 3   1.00
2 1   1.00
2 2   0.00
2 3   0.00
3 1   0.00
3 2   1.00
3 3   0.00
4 1   1.00
4 2   0.00
4 3   0.00
;

'DATA' = DATA

v :=
1 1   60.00
1 2   50.00
1 3   19.00
2 1   75.00
2 2   68.00
2 3   30.00
3 1   69.00
3 2   57.00
3 3   41.00
4 1   79.00
4 2   65.00
4 3   33.00
;

q [*] :=
1  26.00
2  13.00
;

w [*,*,1]
:     1      2      3      :=
1    8.00   6.00   1.00
2    8.00   9.00   3.00
3    9.00   7.00   2.00
4   10.00   8.00   4.00

 [*,*,2]
:    1      2      3      :=
1   4.00   3.00   2.00
2   5.00   4.00   2.00
3   3.00   3.00   4.00
4   3.00   4.00   1.00
;



In [ ]:
ampl.option["solver"] = "highs"
ampl.solve()
ampl.display("")


No variables declared.
file -
line 1
offset 8
syntax error
context:  display  >>> ; <<< 


[AMPL Website](https://ampl.com) | [AMPL Colab](https://colab.ampl.com) | [Community Edition]( https://ampl.com/ce/) | [Twitter](https://twitter.com/AMPLopt) | [LinkedIn](https://www.linkedin.com/company/ampl)

[![Hits](https://h.ampl.com/https://github.com/ampl/amplcolab/blob/master/template/minimal.ipynb)](https://colab.ampl.com)